In [1]:
import numpy as np
import pickle
import pandas as pd

In [2]:
# Carregar variáveis do arquivo pickle
with open('datas.pkl', 'rb') as f:
    data = pickle.load(f)

df = data['df']
rank = data['rank']

In [3]:
df.head

<bound method NDFrame.head of              date home_team away_team  home_score  away_score tournament  \
0      1872-11-30  Scotland   England         0.0         0.0   Friendly   
1      1873-03-08   England  Scotland         4.0         2.0   Friendly   
2      1874-03-07  Scotland   England         2.0         1.0   Friendly   
3      1875-03-06   England  Scotland         2.0         2.0   Friendly   
4      1876-03-04  Scotland   England         3.0         0.0   Friendly   
...           ...       ...       ...         ...         ...        ...   
47121  2024-07-06       NaN       NaN         NaN         NaN  UEFA Euro   
47122  2024-07-06       NaN       NaN         NaN         NaN  UEFA Euro   
47123  2024-07-09       NaN       NaN         NaN         NaN  UEFA Euro   
47124  2024-07-10       NaN       NaN         NaN         NaN  UEFA Euro   
47125  2024-07-14       NaN       NaN         NaN         NaN  UEFA Euro   

             city   country  neutral  
0         Glasgow 

In [4]:
df["date"] = pd.to_datetime(df["date"])

In [5]:
df.isna().sum()

date           0
home_team     15
away_team     15
home_score    51
away_score    51
tournament     0
city           0
country        0
neutral        0
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
df.dtypes

date          datetime64[ns]
home_team             object
away_team             object
home_score           float64
away_score           float64
tournament            object
city                  object
country               object
neutral                 bool
dtype: object

In [8]:
df.sort_values("date").tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
47044,2024-03-26,Tajikistan,Saudi Arabia,1.0,1.0,FIFA World Cup qualification,Dushanbe,Tajikistan,False
47045,2024-03-26,Jordan,Pakistan,7.0,0.0,FIFA World Cup qualification,Amman,Jordan,False
47046,2024-03-26,Bahrain,Nepal,3.0,0.0,FIFA World Cup qualification,Riffa,Bahrain,False
47048,2024-03-26,Bangladesh,Palestine,0.0,1.0,FIFA World Cup qualification,Dhaka,Bangladesh,False
47074,2024-03-26,Finland,Estonia,2.0,1.0,Friendly,Helsinki,Finland,False


In [9]:
df = df[(df["date"] >= "2021-7-11")].reset_index(drop=True)

In [10]:
df.home_team.value_counts()

home_team
United States       37
Mexico              36
Bahrain             34
Qatar               34
Morocco             31
                    ..
Elba Island          1
West Papua           1
Saint Barthélemy     1
Ticino               1
American Samoa       1
Name: count, Length: 249, dtype: int64

In [11]:
rank["rank_date"] = pd.to_datetime(rank["rank_date"])
rank = rank[(rank["rank_date"] >= "2021-7-11")].reset_index(drop=True)

In [12]:
rank["country_full"] = rank["country_full"].str.replace("IR Iran", "Iran").str.replace("Korea Republic", "South Korea").str.replace("USA", "United States")

In [13]:
rank = rank.set_index(['rank_date']).groupby(['country_full'], group_keys=False).resample('D').first().fillna(method='ffill').reset_index()

In [14]:
df_wc_ranked = df.merge(rank[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], 
                        left_on=["date", "home_team"], right_on=["rank_date", "country_full"]).drop(["rank_date", "country_full"], axis=1)

df_wc_ranked = df_wc_ranked.merge(rank[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], 
                                  left_on=["date", "away_team"], right_on=["rank_date", "country_full"], suffixes=("_home", "_away")).drop(["rank_date", "country_full"],
                                                                                                                                           axis=1)

In [15]:
df = df_wc_ranked

In [16]:
def result_finder(home, away):
    if home > away:
        return pd.Series([0, 3, 0])
    if home < away:
        return pd.Series([1, 0, 3])
    else:
        return pd.Series([2, 1, 1])

results = df.apply(lambda x: result_finder(x["home_score"], x["away_score"]), axis=1)

In [17]:
df[["result", "home_team_points", "away_team_points"]] = results

In [18]:
with open('datas.pkl', 'wb') as f:
    pickle.dump({'df': df, 'rank': rank}, f)